# TWAP implementation
_Version 1.0_

In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import pandas as pd

# load our helper functios
from simtools import *

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

matplotlib.rcParams[ 'figure.figsize' ] = ( 16, 9 )

### Load Merged Data

We'll assume we have already done some cleaning by using our Tick Data Setup or similar code

In [ ]:
# set up some variables
tickfile = '../data/TSLA-TAQ-aug19.zip'
test_day = '2019-08-30'

In [ ]:
# load tick data
taq = pd.read_pickle(tickfile)

In [ ]:
len(taq)

In [ ]:
taq.tail()

In [ ]:
# filter on date
day = taq[test_day]

In [ ]:
day.head()

## Main Processing Loop

In [ ]:
# order parameters will go here
order_side = 'b'
quantity = 1000000
minutes_to_trade = 390
slice_size = quantity / minutes_to_trade

# ACCUMULATORS AND MARKET STATE DATA
# initialize some counters
quantity_remaining = quantity
current_bar = 0

# hold completed trades 
trade_data = pd.DataFrame( columns = [ 'price' , 'shares' ], index=day.index )

# init our price and volume variables
[last_price, last_size, bid_price, bid_size, ask_price, ask_size, volume] = np.zeros(7)

# maybe a couple more
[trade_count, quote_count] = [0, 0]

# cumulative volume is nice to have too
cumulative_volume = 0

# main event loop
log_msg('starting main loop')
for index, row in day.iterrows() :
    # get each row: is it a quote or a trade?
    if pd.isna(row.trade_px): # it's a quote
        bid_price = row.bid_px
        bid_size = row.bid_size
        ask_price = row.ask_px
        ask_size = row.ask_size
        quote_count += 1
    else: # it's a trade
        last_price = row.trade_px
        last_size = row.trade_size
        trade_count += 1
        cumulative_volume += row.trade_size
    
    # algo code goes here
    # TODO: FIX ALL THIS
    # figure out which minute we are in after the open
    time_from_open = ( index - pd.Timedelta( hours = 9, minutes = 30 ) )
    minutes_from_open = ( time_from_open.hour * 60 ) + time_from_open.minute
    
    # are we in a new minute?
    if ( minutes_from_open > current_bar ):
        # increment our bar counter
        current_bar = minutes_from_open
        
        # we should do a trade
        if order_side == 'b':
            # buy on the offer
            new_trade_price = ask_price
            new_trade_size = slice_size
            
            # Accumulate the trade per bin
            trade_data.loc[index] = [new_trade_price, new_trade_size]
            
            #print("buy: {0:f} shares at {1:.2f}".format(new_trade_size, new_trade_price))
            
        else: # it's a sell
            # sell on the bid
            new_trade_price = bid_price
            new_trade_size = slice_size
    
            # Accumulate the trade per bin
            trade_data.loc[index] = [new_trade_price, new_trade_size]
            
            #print("sell: {0:f} shares at {1:.2f}".format(new_trade_size, new_trade_price))
    
        # decrement the remaining qty, and bail out if it's zero
        quantity_remaining -= slice_size
    
    # debug
    #print( str(volume) + " " + str(new_trade) + " " + str(quantity_remaining) )
    
    if ( quantity_remaining <= 0 ):
        print(quantity_remaining)
        break
    
log_msg('end main loop')

In [ ]:
print("total quote messages: {:,}".format(quote_count))
print("total trade messages: {:,}".format(trade_count))
print("total shares trades: {:,}".format(cumulative_volume))

### Summary Stats of TWAP Order

In [ ]:
# average trade price
pavg = ( trade_data[ 'price' ] * trade_data[ 'shares' ] ).sum() /  trade_data[ 'shares' ].sum()

#slippage =  ( ( vwap - pavg ) / vwap ) * 10000

# calculate VWAP
#vwap = ( day['close'] * day['volume'] ).sum() / day['volume'].sum()

#slippage =  ( ( vwap - pavg ) / vwap ) * 10000

print( "Avg Price: {:.4f}".format(pavg) )
#print( "VWAP: {:.4f}".format(vwap) )
#print( "Performance vs Day VWAP in bps: {:.4f}".format(slippage) )

In [ ]:
# let's see what we got
fig, ax = plt.subplots(figsize=(16,9))
ax.plot(day.trade_px, 'b-')
ax.plot(trade_data.price, 'r-')
plt.show()

In [ ]:
trade_data[pd.notna(trade_data.price)]